In [1]:
import pandas as pd
import os
import re
import numpy as np

Poniższe 2 komórki wczytują nasz dataset do pliku, następnie wstępnie go przefiltrowują pobierając jedynie 5 miesięcu roku 2010 czyli interesujący nas zakres danych.

In [2]:
# odczytuje scieżke do oryginalnego pliku 
original_data_path = os.getcwd()[:-13] + "sem-5/weather/weather.txt"
print(original_data_path)
with open(original_data_path) as data:
    file = data.readlines()

c:\Users\jakub\Desktop\repo AIBD\AiBD-sem-5/weather/weather.txt


In [3]:
data2010 = []
for elem in file:
    if "2010" in elem[0:21]:
        potrzebne_miesiace = elem[15:17]
        if potrzebne_miesiace == '01' or potrzebne_miesiace == '02' or potrzebne_miesiace == '03' or potrzebne_miesiace == '04' or potrzebne_miesiace == '05':
            data2010.append(elem)
            

Poniższa komórka odpowiedzialna za wyczyszczenie zbioru danych z nieprawidłowych danych.
W tym celu usuwam wszyskie niepoprawne dane tj: S, S-9999, -9999
(Flaga S wg dokumentacji z Global Historical Climatology Network oznacza 'failed spatial consistency check' zatem każda komórka zawierająca w sobie tą daną jest bezużyteczna i należy ją usunąć.)

In [4]:
nazwy_kolumn = ["id", "rok", "miesiac", "nazwa_zmiennych"] + ["dzien{}".format(i) for i in range(1,32)]
stationID = data2010[0][0:11]
wiersze = []
for row in data2010:
    k = len(stationID)
    rok = row[k:k+4]
    miesiac = row[k+4:k+4+2]
    nazwa_zmiennych = row[k+4+2:k+4+2+4]
    day = row[k+4+2+4:].split()
    data_dzienna = row[k+4+2+4:]
    
    zwrot = []
    # Z racji, że podczas dzielenia fragmentu zawierającego temperatury między 
    # kolejnymi pomiarami występowały niejednostajne odstępy między próbkami
    # ograniczyłem dane do 31 próbek.
    if len(day) > 31:
        day = day[:31]

    for el in day:
    # Usuwanie artefaktów
        if el == 'S-9999' or el == 'S' or el == '-9999':
            el = 'inf'
            zwrot.append(el)
        else:
            zwrot.append(el)
    
    wiersze.append([stationID, rok, miesiac, nazwa_zmiennych] + zwrot)

Stworzenie pierwszego DataFramu jeszcze nie zgodnego z zasadami tidy_data

In [5]:
df_not_tidied = pd.DataFrame(wiersze, columns=nazwy_kolumn)
df_not_tidied.head(20)

,id,rok,miesiac,nazwa_zmiennych,dzien1,dzien2,dzien3,dzien4,dzien5,dzien6,...,dzien22,dzien23,dzien24,dzien25,dzien26,dzien27,dzien28,dzien29,dzien30,dzien31
0,MX000017004,2010,01,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,278,inf
1,MX000017004,2010,01,TMIN,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,145,inf
2,MX000017004,2010,01,PRCP,inf,inf,inf,inf,0,inf,...,0,0,inf,0,0,0,0,0,0,inf
3,MX000017004,2010,02,TMAX,inf,273,241,inf,inf,inf,...,inf,299,inf,inf,inf,inf,inf,inf,inf,inf
4,MX000017004,2010,02,TMIN,inf,144,144,inf,inf,inf,...,inf,107,inf,inf,inf,inf,inf,inf,inf,inf
5,MX000017004,2010,02,PRCP,inf,0,0,inf,inf,0,...,inf,0,inf,0,inf,inf,inf,inf,inf,inf
6,MX000017004,2010,03,TMAX,inf,inf,inf,inf,321,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,MX000017004,2010,03,TMIN,inf,inf,inf,inf,142,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,MX000017004,2010,03,PRCP,inf,0,inf,0,0,0,...,inf,0,inf,inf,0,0,inf,inf,0,inf
9,MX000017004,2010,04,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,363,inf,inf,inf,inf


In [6]:
# Procedura opdowiadająca za zapisanie do pliku aktualnego dataframe'a
przefiltrowane = os.getcwd()[:-13] + "sem-5/weather/filtred_data.txt"
print(przefiltrowane)
with open(przefiltrowane, 'w') as out:
    out.write(df_not_tidied.to_csv(index=False))

c:\Users\jakub\Desktop\repo AIBD\AiBD-sem-5/weather/filtred_data.txt


Fragment kodu odpowiadający za utworzenie nowego DataFrame'a wyczyszczonego z błednych lub niepełnych danych.

In [7]:
df_tidied = pd.melt(df_not_tidied, ["id", "rok", "miesiac", "nazwa_zmiennych"])
df_tidied = df_tidied[df_tidied.value != 'inf']
df_tidied = df_tidied.replace(to_replace=["dzien"],value="",regex=True)
df_tidied.sort_values(by=['miesiac','variable'], inplace=True)
df_tidied.head(40)

,id,rok,miesiac,nazwa_zmiennych,variable,value
167,MX000017004,2010,01,PRCP,12,0
197,MX000017004,2010,01,PRCP,14,0
212,MX000017004,2010,01,PRCP,15,0
257,MX000017004,2010,01,PRCP,18,0
272,MX000017004,2010,01,PRCP,19,0
287,MX000017004,2010,01,PRCP,20,0
302,MX000017004,2010,01,PRCP,21,0
317,MX000017004,2010,01,PRCP,22,0
332,MX000017004,2010,01,PRCP,23,0
362,MX000017004,2010,01,PRCP,25,0


DataFrame nie spełnia jeszcze zasad tidy data, dlatego połączyłem kolumny odpowiadające za rok, miesiac, dzien w jedną kolumnę o nazwie date.


In [8]:
df_tidied = df_tidied.assign(date = df_tidied.rok.astype(str) + "-" + df_tidied.miesiac.astype(str) + "-" + df_tidied.variable.astype(str)[:] )
df_tidied.pop("rok")
df_tidied.pop("variable")
df_tidied.pop("miesiac")

df_tidied = df_tidied[["id","date","nazwa_zmiennych","value"]]

df_tidied.head(40)

,id,date,nazwa_zmiennych,value
167,MX000017004,2010-01-12,PRCP,0
197,MX000017004,2010-01-14,PRCP,0
212,MX000017004,2010-01-15,PRCP,0
257,MX000017004,2010-01-18,PRCP,0
272,MX000017004,2010-01-19,PRCP,0
287,MX000017004,2010-01-20,PRCP,0
302,MX000017004,2010-01-21,PRCP,0
317,MX000017004,2010-01-22,PRCP,0
332,MX000017004,2010-01-23,PRCP,0
362,MX000017004,2010-01-25,PRCP,0


In [9]:
tymczasowe = os.getcwd()[:-13] + "sem-5/weather/temp_data.txt"
print(tymczasowe)
with open(tymczasowe, 'w') as file:
    file.write(df_tidied.to_csv(index=False))

c:\Users\jakub\Desktop\repo AIBD\AiBD-sem-5/weather/temp_data.txt


Dobrym sposobem na lepsze przedstawienie naszych danych będzie podzielenie kolumny nazwa_zmiennych, która zawiera 3 typy pomiarów do osobnych kolumn kolejno: TMAX, TMIN, PRCP.

In [10]:
# Rozdzielenie TMAX, TMIN, PRCP do osobnych kolumn
df = pd.pivot_table(df_tidied, index=['id', 'date'], columns=['nazwa_zmiennych'], values='value', aggfunc='last')
df.dropna(inplace=True)
df.reset_index(drop=False, inplace=True)
df.head(10)

nazwa_zmiennych,id,date,PRCP,TMAX,TMIN
0,MX000017004,2010-01-30,0,278,145
1,MX000017004,2010-02-11,0,297,134
2,MX000017004,2010-02-2,0,273,144
3,MX000017004,2010-02-23,0,299,107
4,MX000017004,2010-02-3,0,241,144
5,MX000017004,2010-03-10,0,345,168
6,MX000017004,2010-03-16,0,311,176
7,MX000017004,2010-03-5,0,321,142
8,MX000017004,2010-04-27,0,363,167
9,MX000017004,2010-05-27,0,332,182


Zapis danych końcowych do pliku.

In [11]:
finalne = os.getcwd()[:-13] + "sem-5/weather/final_data.txt"
print(tymczasowe)
with open(finalne, 'w') as file:
    file.write(df.to_csv(index=False))

c:\Users\jakub\Desktop\repo AIBD\AiBD-sem-5/weather/temp_data.txt
